# Sayou Connector Quick Start

**Sayou Fabric**의 데이터 수집기인 `sayou-connector`의 기본 사용법을 다룹니다.
이 노트북에서는 다음 세 가지 시나리오를 시연합니다.

1. **Local File Scan:** 로컬 폴더의 문서 파일 수집
2. **SQLite Scan:** 데이터베이스의 대량 데이터 페이지네이션 수집
3. **Web Crawling:** 웹 뉴스 기사 수집 및 링크 탐색

In [ ]:
import os
import sqlite3
import logging
import shutil

# 사유존 커넥터 임포트
from sayou.connector.pipeline import ConnectorPipeline

# 로그 레벨 설정 (INFO로 설정하여 주요 흐름만 확인)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

print("Import Successful!")

## 1. 초기화 (Initialization)

파이프라인을 생성하고 초기화합니다. 이 과정에서 기본 제공되는 플러그인(File, SQLite, Web)들이 자동으로 등록됩니다.

In [ ]:
# 오케스트레이터 초기화
pipeline = ConnectorPipeline()
pipeline.initialize()

## 2. 시나리오: Local File Scan

특정 폴더에 있는 파일들을 재귀적으로 탐색하여 읽어옵니다.
테스트를 위해 임시 파일들을 먼저 생성하겠습니다.

In [ ]:
# [Helper] 테스트용 더미 파일 생성
TEST_ROOT = "test_docs_notebook"

if os.path.exists(TEST_ROOT):
    shutil.rmtree(TEST_ROOT)
os.makedirs(TEST_ROOT)

with open(f"{TEST_ROOT}/readme.txt", "w", encoding="utf-8") as f:
    f.write("Welcome to Sayou Fabric!")
with open(f"{TEST_ROOT}/config.json", "w", encoding="utf-8") as f:
    f.write('{"version": "0.1.0"}')
    
print(f"Created dummy files in '{TEST_ROOT}'")

In [ ]:
# 파이프라인 실행 (전략: file)
# extensions=['.txt']로 설정하여 txt 파일만 수집합니다.

file_packets = pipeline.run(
    source=TEST_ROOT,
    strategy="file",
    extensions=[".txt"],  # json 파일은 무시됨
    recursive=True
)

print("=== File Scan Results ===")
for i, packet in enumerate(file_packets):
    if packet.success:
        # FileFetcher는 바이너리(bytes)를 반환하므로 디코딩
        content = packet.data.decode("utf-8")
        print(f"[{i}] File: {packet.task.meta['filename']}")
        print(f"    Path: {packet.task.uri}")
        print(f"    Content: {content}")
    else:
        print(f"[{i}] Error: {packet.error}")

## 3. 시나리오: SQLite DB Scan

데이터베이스에서 대량의 데이터를 가져옵니다.
`batch_size`를 설정하면, Generator가 자동으로 `LIMIT/OFFSET` 쿼리를 생성하여 끊어서 가져옵니다.

In [ ]:
# [Helper] 테스트용 DB 생성
DB_PATH = "test_users_notebook.db"

if os.path.exists(DB_PATH):
    os.remove(DB_PATH)

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()
cur.execute("CREATE TABLE users (id INTEGER, name TEXT)")

# 25개의 데이터 생성
for i in range(25):
    cur.execute("INSERT INTO users VALUES (?, ?)", (i, f"User_{i:03d}"))
conn.commit()
conn.close()

print(f"Created dummy DB at '{DB_PATH}' with 25 rows.")

In [ ]:
# 파이프라인 실행 (전략: sqlite)
# batch_size=10으로 설정 (총 25개이므로 3번의 Fetch 발생 예상: 10개, 10개, 5개)

db_packets = pipeline.run(
    source=DB_PATH,
    strategy="sqlite",
    query="SELECT * FROM users",
    batch_size=10
)

print("=== DB Scan Results ===")
for i, packet in enumerate(db_packets):
    if packet.success:
        rows = packet.data
        offset = packet.task.meta.get('offset')
        print(f"Batch {i+1} (Offset {offset}): Fetched {len(rows)} rows")
        
        # 첫 번째 배치의 데이터 샘플 확인
        if i == 0 and rows:
            print(f"    Sample Row: {rows[0]}")
    else:
        print(f"Batch {i+1} Failed: {packet.error}")

## 4. 시나리오: Web Crawling

웹 페이지를 수집하고, 페이지 내의 링크를 찾아 다음 수집 대상을 자동으로 확장합니다.
(네트워크 환경에 따라 실패할 수 있습니다.)

In [ ]:
# 다음(Daum) 뉴스 테크 섹션 예시
TARGET_URL = "https://news.daum.net/tech"

# 링크 패턴: 기사 본문 URL만 수집 (정규식)
LINK_PATTERN = r"https://v\.daum\.net/v/\d+"

# CSS 선택자: 제목과 본문 추출
SELECTORS = {
    "title": ".head_view",     # (사이트 구조 변경 시 수정 필요)
    "content": ".article_view"
}

print(f"Starting crawl on: {TARGET_URL}")

try:
    web_packets = pipeline.run(
        source=TARGET_URL,
        strategy="requests",
        link_pattern=LINK_PATTERN,
        selectors=SELECTORS,
        max_depth=1  # 1단계 깊이까지만 탐색
    )

    count = 0
    for packet in web_packets:
        if not packet.success:
            continue
        
        # 데이터 확인 (메타데이터 제외)
        data = packet.data
        clean_data = {k: v[:50]+"..." for k, v in data.items() if not k.startswith("__")}
        
        # 제목이 추출된 경우만 출력
        if clean_data.get("title"):
            print(f"\n[{count}] Found Article: {packet.task.uri}")
            print(f"    Title: {clean_data.get('title')}")
            count += 1
            
            if count >= 3:
                print("\n... (데모를 위해 3개만 출력하고 중단합니다)")
                break

except Exception as e:
    print(f"Web Crawling Skipped: {e}")

In [ ]:
# [Cleanup] 테스트 파일 정리
if os.path.exists(TEST_ROOT):
    shutil.rmtree(TEST_ROOT)
if os.path.exists(DB_PATH):
    os.remove(DB_PATH)

print("Cleanup complete.")